In [1]:
# import pandas as pd
# df = pd.read_csv('../csv_data/demesne_v1.csv')
# df

In [60]:
from pdf2df import convert_to_df
df = convert_to_df('../pdf_data/Kilkenny/Kilkenny Welding Supplies Limited_Invoice_PL017892_211220094854109.pdf')
df

{'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Rave Reports (http://www.nevrona.com/rave)', 'producer': 'Nevrona Designs', 'creationDate': 'D:20201221094854Z', 'modDate': 'D:20210115092842Z', 'trapped': '', 'encryption': None}


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
0,Invoice,511,19,571,41,0,0,1
1,Kilkenny,19,21,80,40,0,1,1
2,Welding,84,21,142,40,0,1,1
3,Supplies,146,21,205,40,0,1,1
4,Limited,209,21,262,40,0,1,1
...,...,...,...,...,...,...,...,...
226,the,353,724,364,735,51,0,1
227,supplier,367,724,395,735,51,0,1
228,is,397,724,402,735,51,0,1
229,discharged.,405,724,446,735,51,0,1


In [61]:
df= df.sort_values(by=['Y1','X1']).reset_index(drop=True)


In [62]:
def rearrange(df):
    for i in range(1, len(df)):
        temp=df.loc[i,'Y1'] - df.loc[i-1, 'Y1']
        if -5 < temp < 10:
            df.loc[i,'Y1'] = df.loc[i-1,'Y1']
    
    return df.sort_values(by=['Y1','X1']).reset_index(drop=True)
    

In [63]:
df = rearrange(df)

In [64]:
def find_text(temp_df, text,case=False):
    location = temp_df[temp_df['TEXT'].str.contains(text,case=case, na=False)].index.tolist()
    return location

def find_next_word(temp_df, text, nearest_text, case=False):
    pos = find_text(temp_df,text,case=case)
    pos = [item +1 for item in pos]
#     print(pos)
    new_pos = list()
    for item in pos:
#         temp = df.loc[]
        temp_value= temp_df.loc[item]['TEXT']
#         print(temp_value.lower())
       
        if temp_value.lower().__contains__(nearest_text.lower()):
            new_pos.append(item-1)
        
    return new_pos

In [65]:
pos = find_text(df, 'Number')
invoice_no  =df[pos[0]+1:pos[0]+2]['TEXT'].values[0]
invoice_no


'PL017892'

In [66]:
pos = find_text(df, 'Date')
invoice_date = df[pos[0]+1:pos[0]+2]['TEXT'].values[0]
invoice_date

'07/12/2020'

In [67]:
find_text(df,'VAT')

[28, 159, 164, 175]

In [68]:
find_next_word(df,'VAT','Analysis')

[159]

In [69]:
table_start = find_text(df, 'Unit')
print(table_start)
table_end = find_next_word(df,'VAT','Analysis')
print(table_end)
table_data = df[table_start[0]:table_end[0]]
table_data

[85]
[159]


,TEXT,X1,Y1,X2,Y2,block_no,line_no,page_no
85,Unit,392,250,413,264,10,1,1
86,Price,416,250,441,264,10,1,1
87,Discount,455,250,499,264,10,0,1
88,%,501,250,513,264,10,0,1
89,Amount,514,250,553,264,8,3,1
...,...,...,...,...,...,...,...,...
154,20.00,472,389,497,402,30,3,1
155,7.55,536,389,555,402,30,1,1
156,0.37kg,97,401,127,414,29,0,1
157,/,130,401,134,414,29,0,1


In [70]:
from pprint import pprint
msg = dict()
result = list()
for index, row in table_data.iterrows():
    if  row['X1'] < 30:
#         print(row['TEXT'])
        msg = dict()
        msg['part'] = row['TEXT']
        msg['desc'] = ''
    
    
    elif ('desc' in msg) and (90 < row['X1'] < 300):
#         print(row['TEXT'])
        msg['desc'] = msg['desc'] +" "+ row['TEXT']
        msg['qty'] = ''
       
    elif ('qty' in msg) and (340 < row['X1'] < 374):
#         print(row['TEXT'])
        msg['qty'] = row['TEXT']
        msg['unit'] = ''

    if ('unit' in msg) and (400  < row['X1'] < 438):
#         print(row['TEXT'])
        msg['unit'] = row['TEXT']
        msg['disc'] = 0
        msg['total'] = 0 
        
    if 'disc' in msg and 470  < row['X1'] < 500:
        msg['disc'] = row['TEXT']
        msg['total'] = 0
        
    if 'total' in msg and 520 < row['X1'] < 560:
        msg['total'] = row['TEXT']
        pprint(msg)
        result.append(msg)
        msg = dict()
    

{'desc': ' 70SQ WELDING CABLE',
 'disc': '35.00',
 'part': 'CABWEL070',
 'qty': '12.00',
 'total': '125.97',
 'unit': '16.15'}
{'desc': ' 600AMP EARTH CLAMP SCREW TYPE',
 'disc': '20.00',
 'part': 'EARSCR600',
 'qty': '1.00',
 'total': '11.70',
 'unit': '14.63'}
{'desc': ' 70-95SQ Dinze Plug',
 'disc': '30.00',
 'part': 'DINPLU070',
 'qty': '1.00',
 'total': '8.68',
 'unit': '12.40'}
{'desc': ' TIP 1.2MM M8 PKT 10',
 'disc': '20.00',
 'part': 'FR420001646710',
 'qty': '1.00',
 'total': '36.10',
 'unit': '45.12'}
{'desc': ' SPATTER GUARD MTW500I PKT 5',
 'disc': '20.00',
 'part': 'FR42040508545',
 'qty': '0.20',
 'total': '3.46',
 'unit': '21.64'}
{'desc': ' CONICAL NOZZLE MTW500I PKT 5',
 'disc': '20.00',
 'part': 'FR42000140505',
 'qty': '0.20',
 'total': '7.55',
 'unit': '47.22'}


In [71]:
result.__len__()

6